In [1]:
%reset -f

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import tools
import time 

In [2]:
# Parameters to tweak
lr = 1e-3
batchSize = 64
epochs = 30
loss_fn = nn.CrossEntropyLoss()

In [3]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

# Create 64 batches
train_dataloader = DataLoader(training_data, batch_size=batchSize)
test_dataloader = DataLoader(test_data, batch_size=batchSize)
len(train_dataloader)

938

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
def train_data(model, optimizer):
    for xb, yb in train_dataloader:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = loss.item()
    #print(f"Train loss: {loss:>7f}")

In [7]:
def test_data(model):
    num_batches = len(test_dataloader)
    size = len(test_dataloader.dataset)
    test_loss, corrects = 0, 0

# test_results = [(np.argmax(self.feedforward(x)), y)
#                        for (x, y) in test_data]

    with torch.no_grad():
        for xb, yb in test_dataloader:
            preds = model(xb)
            test_loss += loss_fn(preds, yb).item()
            corrects += (preds.argmax(1) == yb).type(torch.float).sum().item()

    test_loss /= num_batches
    # test_loss = lo
    corrects /= size

    print(f"Accuracy: {(100*corrects):>0.1f}%, Avg loss: {test_loss:>8f}")
    return corrects

In [8]:
epoch_pro = []
epoch_performance = []
for n in range(0, 33):
    print(f"Started run {n} / {33}")

    model = NeuralNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for t in range(epochs):
        start_time = time.time()

        train_data(model, optimizer)
        epoch_pro.append(test_data(model))
        epoch_performance.append(time.time()-start_time)

Started run 0 / 33
Accuracy: 64.6%, Avg loss: 2.036152
Accuracy: 74.7%, Avg loss: 1.715983
Accuracy: 78.6%, Avg loss: 1.375053
Accuracy: 81.1%, Avg loss: 1.102060
Accuracy: 82.8%, Avg loss: 0.914699
Accuracy: 84.3%, Avg loss: 0.788704
Accuracy: 85.2%, Avg loss: 0.701064
Accuracy: 86.0%, Avg loss: 0.637357
Accuracy: 86.7%, Avg loss: 0.589176
Accuracy: 87.0%, Avg loss: 0.551528
Accuracy: 87.4%, Avg loss: 0.521328
Accuracy: 87.8%, Avg loss: 0.496588
Accuracy: 88.1%, Avg loss: 0.475965
Accuracy: 88.4%, Avg loss: 0.458519
Accuracy: 88.7%, Avg loss: 0.443573
Accuracy: 89.0%, Avg loss: 0.430625
Accuracy: 89.1%, Avg loss: 0.419298
Accuracy: 89.2%, Avg loss: 0.409301
Accuracy: 89.5%, Avg loss: 0.400409
Accuracy: 89.6%, Avg loss: 0.392440
Accuracy: 89.7%, Avg loss: 0.385251
Accuracy: 89.8%, Avg loss: 0.378726
Accuracy: 89.9%, Avg loss: 0.372771
Accuracy: 90.0%, Avg loss: 0.367308
Accuracy: 90.1%, Avg loss: 0.362271
Accuracy: 90.2%, Avg loss: 0.357607
Accuracy: 90.3%, Avg loss: 0.353270
Accuracy:

In [ ]:
tools.saveResult(epochs,lr,"crossentropyloss",batchSize,epoch_pro,epoch_performance, "model_cross_entropy_relu_mandag")